In [1]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

Image.MAX_IMAGE_PIXELS = None
electricityBillsFolder = './electricity'
plotsFolder = './GraphPlots/ElectricityPlots'
mergedPlots = './GraphPlots/ElectricityPlots/MergedPlots'

regions = ['SE1', 'SE2', 'SE3', 'SE4']

# Ensure directories exist
os.makedirs(plotsFolder, exist_ok=True)
os.makedirs(mergedPlots, exist_ok=True)
for region in regions:
    os.makedirs(os.path.join(plotsFolder, region), exist_ok=True)


In [2]:
def plotHourlyData(df, df_name, time_column, value_column):
    try:
        regionName = df_name.split('_')[0]
        fileName = df_name.split('_')[-1]
        targetFile = os.path.join(plotsFolder, regionName, fileName)

        # Convert to string and drop NaN values
        df[time_column] = df[time_column].astype(str)
        df = df.dropna(subset=[time_column])

        # Extract the start time and convert to datetime
        df['Timestamp'] = pd.to_datetime(df[time_column].str.split(' - ').str[0], format='%d.%m.%Y %H:%M', errors='coerce')

        df = df.dropna(subset=['Timestamp'])

        df[value_column] = pd.to_numeric(df[value_column].replace('n/e', pd.NA), errors='coerce')

        # Drop rows where the value column is NaN (which means we had an invalid value like 'n/e')
        df = df.dropna(subset=[value_column])

        # Sort by time
        df = df.sort_values(by='Timestamp')

        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(df['Timestamp'], df[value_column], marker='o', linestyle='-', color='b', label=str(value_column))

        # Formatting
        plt.xlabel("Time (Hourly)")
        plt.ylabel(value_column)
        plt.title(fileName.replace('.png', ''))
        plt.xticks(rotation=45)
        plt.legend()
        plt.grid(True)

        # Save plot
        print(f"Saving plot to {targetFile}...")
        plt.savefig(targetFile, dpi=1200, bbox_inches='tight')
        plt.close()

        print(f"✅ Plot saved: {targetFile}")

    except Exception as e:
        print(f"Plot Hourly Rates Error: {e}")


def readCsv(filePath: str):
    try:
        df = pd.read_csv(filePath)
        return df
    except Exception as e:
        print(f"Read CSV Error: {e}")
        return None

def mergePlots(sourceFolder, destinationFileName, horizontalFlag):
    images = [Image.open(os.path.join(sourceFolder, f)) for f in sorted(os.listdir(sourceFolder)) if f.endswith(".png")]
    
    if not images:
        print("❌ No images found in the folder.")
        return

    if horizontalFlag:
        total_width = sum(img.width for img in images)
        max_height = max(img.height for img in images)
        merged_image = Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for img in images:
            merged_image.paste(img, (x_offset, 0))
            x_offset += img.width
    else:
        max_width = max(img.width for img in images)
        total_height = sum(img.height for img in images)
        merged_image = Image.new('RGB', (max_width, total_height))
        y_offset = 0
        for img in images:
            merged_image.paste(img, (0, y_offset))
            y_offset += img.height
    
    merged_image.save(destinationFileName, format='PNG')
    print(f"✅ Merged image saved to {destinationFileName}")


In [3]:
electricity_files = [f for f in os.listdir(electricityBillsFolder) if f.endswith('.csv')]

for file in electricity_files:
    parts = file.replace('_entsoe.csv', '').split('_')
    if len(parts) == 2:
        zone, year = parts[0], parts[1]  # Extract zone & year
        filePath = os.path.join(electricityBillsFolder, file)
        
        print(f"\n📈 Processing: {file}")
        df = readCsv(filePath)  # Read CSV
        
        if df is not None and not df.empty:
            plotHourlyData(df, f"{zone}_{year}.png", 'MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]')
            print(f"✅ Saved plot for {zone} - {year}")
        else:
            print(f"❌ Skipping {file} due to empty or unreadable data.")





📈 Processing: SE1_2016_entsoe.csv
Saving plot to ./GraphPlots/ElectricityPlots\SE1\2016.png...
✅ Plot saved: ./GraphPlots/ElectricityPlots\SE1\2016.png
✅ Saved plot for SE1 - 2016

📈 Processing: SE1_2017_entsoe.csv
Saving plot to ./GraphPlots/ElectricityPlots\SE1\2017.png...
✅ Plot saved: ./GraphPlots/ElectricityPlots\SE1\2017.png
✅ Saved plot for SE1 - 2017

📈 Processing: SE1_2018_entsoe.csv
Saving plot to ./GraphPlots/ElectricityPlots\SE1\2018.png...
✅ Plot saved: ./GraphPlots/ElectricityPlots\SE1\2018.png
✅ Saved plot for SE1 - 2018

📈 Processing: SE1_2019_entsoe.csv
Saving plot to ./GraphPlots/ElectricityPlots\SE1\2019.png...
✅ Plot saved: ./GraphPlots/ElectricityPlots\SE1\2019.png
✅ Saved plot for SE1 - 2019

📈 Processing: SE1_2020_entsoe.csv
Saving plot to ./GraphPlots/ElectricityPlots\SE1\2020.png...
✅ Plot saved: ./GraphPlots/ElectricityPlots\SE1\2020.png
✅ Saved plot for SE1 - 2020

📈 Processing: SE1_2021_entsoe.csv
Saving plot to ./GraphPlots/ElectricityPlots\SE1\2021.png...

In [4]:
for region in regions:
    regionFolder = os.path.join(plotsFolder, region)
    dest = os.path.join(mergedPlots, f"{region}.png")
    mergePlots(regionFolder, dest, False)


✅ Merged image saved to ./GraphPlots/ElectricityPlots/MergedPlots\SE1.png
✅ Merged image saved to ./GraphPlots/ElectricityPlots/MergedPlots\SE2.png
✅ Merged image saved to ./GraphPlots/ElectricityPlots/MergedPlots\SE3.png
✅ Merged image saved to ./GraphPlots/ElectricityPlots/MergedPlots\SE4.png
